*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


### Imports

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

2.8.0


### Downloading

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-04-19 17:19:45--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.06s   

2022-04-19 17:19:46 (6.04 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-04-19 17:19:46--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
features_names=['indicates', 'text']
df_train = pd.read_table(train_file_path, header=0, names=features_names)
df_test = pd.read_table(test_file_path, header=0, names=features_names)

#### Processing data

##### Transforming of categorical labels

In [4]:
# Changing categorical labels to numbers (0,1)
# Changing categorical labels to numbers (0,1)
df_train["indicates"] = [1 if each == 'spam' else 0 for each in df_train["indicates"]] # FOR FUTURE: it did not work to compare using "" instead of ''
df_test["indicates"] = [1 if each == 'spam' else 0 for each in df_test["indicates"]]

##### Finding the unique vocabs in the train and test dataset

In [5]:
# merging training and testing dataset
total_data_frames = [df_train, df_test]
total_data = pd.concat(total_data_frames)

In [6]:
total_data

,indicates,text
0,0,you can never do nothing
1,0,"now u sound like manky scouse boy steve,like! ..."
2,0,mum say we wan to go then go... then she can s...
3,0,never y lei... i v lazy... got wat? dat day ü ...
4,0,in xam hall boy asked girl tell me the startin...
...,...,...
1386,0,true dear..i sat to pray evening and felt so.s...
1387,0,"what will we do in the shower, baby?"
1388,0,where are you ? what are you doing ? are yuou ...
1389,1,ur cash-balance is currently 500 pounds - to m...


In [8]:
vocabs=set()
MAX_LEN=0
for value in total_data['text'].values:
    tokens=value.split()
    if (len(tokens) > MAX_LEN):
      MAX_LEN=len(tokens)
    vocabs.update(tokens)

In [9]:
VOCAB_SIZE=len(vocabs) # count of unique vocabs

##### Encoding and padding the text

In [13]:
# Define the tokenizer 
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<UNK>')
tokenizer.fit_on_texts(vocabs)
# encoding and padding training data
Training_Sequences = tokenizer.texts_to_sequences(df_train['text'])
Training_pad = pad_sequences(Training_Sequences, maxlen=MAX_LEN)
# encoding and padding testing data
Testing_Sequences = tokenizer.texts_to_sequences(df_test['text'])
Testing_pad = pad_sequences(Testing_Sequences, maxlen=MAX_LEN)

In [14]:
Training_Sequences_padded = np.asarray(Training_pad)
Testing_Sequences_padded = np.asarray(Testing_pad)
Training_Labels = np.asarray(df_train['indicates'])
Testing_Lables = np.asarray(df_test['indicates'])

### Defining, compiling and training the model

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [16]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [17]:
history = model.fit(Training_Sequences_padded, Training_Labels, validation_data=(Testing_Sequences_padded, Testing_Lables), epochs=5)

Epoch 1/5
131/131 [==============================] - 20s 78ms/step - loss: 0.2421 - accuracy: 0.9179 - val_loss: 0.0582 - val_accuracy: 0.9806
Epoch 2/5
131/131 [==============================] - 9s 71ms/step - loss: 0.0329 - accuracy: 0.9897 - val_loss: 0.0804 - val_accuracy: 0.9806
Epoch 3/5
131/131 [==============================] - 9s 71ms/step - loss: 0.0125 - accuracy: 0.9969 - val_loss: 0.0469 - val_accuracy: 0.9871
Epoch 4/5
131/131 [==============================] - 9s 70ms/step - loss: 0.0048 - accuracy: 0.9988 - val_loss: 0.0435 - val_accuracy: 0.9856
Epoch 5/5
131/131 [==============================] - 9s 70ms/step - loss: 0.0020 - accuracy: 0.9993 - val_loss: 0.0437 - val_accuracy: 0.9856


### Predicting and testing

In [18]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  Test = tokenizer.texts_to_sequences([pred_text])[0]
  Test_padded = pad_sequences([Test], maxlen=MAX_LEN)#, padding=PADD_TYPE, truncating=TRUNC_TYPE)
  Test_padded = np.asarray(Test_padded)
  #pred_text = encoder.encode(pred_text) # encoding the text to be predicted
  #pred_text = tf.cast(pred_text, tf.float32) # Casts a tensor to a float32
  prediction = model.predict(Test_padded)#tf.expand_dims(pred_text, 0)).tolist()
  #print(prediction)
  if prediction[0][0] < 0.5:
    return[prediction[0][0],'ham']
  else:
    return[prediction[0][0],'spam']
  #return (prediction)
pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[-9.112641, 'ham']


In [20]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

You passed the challenge. Great job!
